# Scraping Tweets (UK)

This notebook consists the data scraping part of the project, namely, scraping UK tweets from Twitter using the Twitter API.

In [1]:
# Import dependancies
import tweepy
import re
import requests
import pandas as pd
from tqdm.auto import tqdm
import numpy as np
import pickle
import time
import itertools

def get_tweets(tweet_ids):
    '''
    Gets tweets from tweet_ids and for the ones that it cannot find, save to error list and return it

            Parameters:
                    tweet_ids (list): List of tweet IDs

            Returns:
                    tweets (list): List of lists of tweet IDs
                    errors (list): List of lists of tweet_IDs that it could not find
    '''
    # Parameters for Twitter API client **Confidential
    client = tweepy.Client(
        bearer_token="AAAAAAAAAAAAAAAAAAAAAJhtiwEAAAAAnSH7LB8LCFaBffd9NrgflhozHQ8%3Dgl3augKz2Chu0iqikKNcszMsnvQxzPURyiCyQx0DjNwhDDpddO",
        consumer_key="x0ZhcTVGaRmabZ0wzw7cA6TFt",
        consumer_secret="CuI7c9JT8d8PkqkjK73JmTmSiowSQVLpQSAD7KlPwX2DfvKsm7",
        access_token="1454490866597441542-qkkzuAWHrfd84M2DDSBiHQtxjGi6GV",
        access_token_secret="n9X2PFuzNlxfgJdb54Uqkj0SzLIxQDWqW50ZYFZvfdPBi",
    )

    response = client.get_tweets(ids=tweet_ids)
    tweets = [tweet.text for tweet in tqdm(response.data)]
    errors = [error['value'] for error in response.errors]
    
    return tweets, errors

In [13]:
# Load data
data_df = pd.read_csv("files/project_tweet_data_final_UK.csv",index_col=0)

In [14]:
# Inspect first five elements
data_df.head(5)

,tweet_ID,user_id,keywords,tweet_timestamp,valence_intensity,fear_intensity,anger_intensity,happiness_intensity,sadness_intensity,sentiment,emotion
0,1234289474479239168,303154079,covid,2020-03-02 01:28:37,0.380,0.516,0.454,0.207,0.458,-1,F
1,1234290271774543872,248819855,covid,2020-03-02 01:31:47,0.380,0.611,0.412,0.218,0.460,-1,F
2,1234291397202776065,248819855,covid,2020-03-02 01:36:15,0.408,0.505,0.422,0.218,0.417,-1,F
3,1234292673260879872,447663369,corona,2020-03-02 01:41:20,0.448,0.566,0.419,0.267,0.415,-1,F
4,1234293534498414592,406164260,covid,2020-03-02 01:44:45,0.575,0.465,0.306,0.390,0.337,1,H


In [15]:
# Inspect shape
data_df.shape

(733334, 11)

In [17]:
# Get list of tweet IDs to scrape
tweet_IDs = data_df["tweet_ID"].tolist()

# Take the first 100 tweets (for testing)
tweet_IDs_test = tweet_IDs[:100]
tweets_array = np.asarray(tweet_IDs_test)

# Partition into 10 equally sized arrays
split_array = np.array_split(tweets_array, 10)

for i in range(len(split_array)):
    split_array[i] = list(split_array[i])

In [19]:
# Inspect length
print(len(np.asarray(tweet_IDs)))

733334


In [27]:
# Get array with tweet IDs
tweets_array_final = np.asarray(tweet_IDs)

# Split into equally sized partitions
split_array_final = np.array_split(tweets_array_final, 7334)

for i in tqdm(range(len(split_array_final))):
    split_array_final[i] = list(split_array_final[i])

# Save list of tweet IDs to pickle file
with open('files/pickles/list_tweet_ids_UK.pkl', 'wb') as f:
    pickle.dump(split_array_final, f)

# Inspect list
print(len(split_array_final))

# Inspect final element
print(len(split_array_final[-1]))

  0%|          | 0/7334 [00:00<?, ?it/s]

7334
99


In [32]:
%%time
tweet_list = []
error_list = []

## Retrieve tweets from tweet IDs ##
for i in tqdm(range(len(split_array_final))):
    try:
        tweets, errors = get_tweets(split_array_final[i])   
    except:
        time.sleep(900) # Twitter API only allows for max 900 calls per 15 min. (900 secs) timeframe; sleep and continue
        tweets, errors = get_tweets(split_array_final[i])  

    tweet_list.append(tweets)
    error_list.append(errors)

# Save tweets
with open('files/pickles/tweets_UK.pkl', 'wb') as f:
    pickle.dump(tweet_list, f)

# Save non-existent tweets
with open('files/pickles/errors_UK.pkl', 'wb') as f:
    pickle.dump(error_list, f)

  0%|          | 0/7334 [00:00<?, ?it/s]

  0%|          | 0/81 [00:00<?, ?it/s]

  0%|          | 0/73 [00:00<?, ?it/s]

  0%|          | 0/72 [00:00<?, ?it/s]

  0%|          | 0/78 [00:00<?, ?it/s]

...

  0%|          | 0/79 [00:00<?, ?it/s]

  0%|          | 0/77 [00:00<?, ?it/s]

  0%|          | 0/89 [00:00<?, ?it/s]

  0%|          | 0/85 [00:00<?, ?it/s]

Wall time: 7h 18min 58s


In [33]:
# Open tweets list pickle for inspection
with open('files/pickles/tweets_UK.pkl', 'rb') as f:
    mynewlist = pickle.load(f)

# Inspect length
print(len(mynewlist))

7334


In [35]:
# Inspect first tweet
print(mynewlist[0])

['HEALTH EMERGENCY / INTERNATIONAL COVID-19 CORONAVIRUS OUTBREAK: https://t.co/HPvNkqAMH0', 'HEALTH EMERGENCY / INTERNATIONAL COVID-19 CORONAVIRUS OUTBREAK: COVID-19 global cases: Johns Hopkins CSSE online electronic dashboard data as of 2nd March 2020:   https://t.co/QdyloFhT9i', 'Corruption scaring corona virus from Africa &gt;&gt;&gt;', "Starting up with a cold and lightly concerned that it's Corona-chan.\nAlso got a job interview on Thursday, so I'm gonna be spreading it through two major cities.\nI either get the job or you get the Chinese superflu, pick one.", 'The UK government is considering all options, if the coronavirus outbreak continues to escalate. @darrenadam asks: What sacrifices are you willing to accept in combating COVID-19? https://t.co/6oPYBXUoCg', 'USA the land of liberty charges $3000 to get tested for Corona Virus.\nIf this takes off America is as secure as Europe during the black plague. https://t.co/HQChq6ZjJT', 'HEALTH EMERGENCY / COVID-19 CORONAVIRUS OUTBREA

In [38]:
# Check overall number of successfully scraped tweets
final_list = list(itertools.chain.from_iterable(mynewlist))
print(len(final_list))

593237


In [39]:
# Check final tweet
print(final_list[-1])

Israelis protest against Netanyahu, gov't handling of COVID-19 crisis&gt;https://t.co/9Gb7oqkPGs&gt;Brazil’s far-right president, the self-declared homophobe Jair Bolsonaro,https://t.co/uhxYLgZQDL https://t.co/zIVTirnt50
